In [1]:
# DAY 01
nums = [int(x) for x in open("../data/01.txt").readlines()]

def sliding_window(arr, n):
    for i in range(0, len(arr) - n + 1):
        yield arr[i:i+n]

res1 = sum(x[1] > x[0] for x in sliding_window(nums, 2))
res2 = sum(x[1] > x[0] for x in sliding_window(
    [sum(y) for y in sliding_window(nums, 3)], 
    2))

print(f'Answer 1: {res1}\nAnswer 2: {res2}')

Answer 1: 1557
Answer 2: 1608


In [2]:
# DAY 02
from functools import reduce

commands = [(x.split()[0], int(x.split()[1])) 
             for x in open("../data/02.txt").readlines()]

def step1(pos, command):
    (x, depth) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth - offs)
    elif cmd == 'down':
        return (x, depth + offs)
    elif cmd == 'forward':
        return (x + offs, depth)
    
def step2(pos, command):
    (x, depth, aim) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth, aim - offs)
    elif cmd == 'down':
        return (x, depth, aim + offs)
    elif cmd == 'forward':
        return (x + offs, depth + aim * offs, aim)
        
pos1 = reduce(step1, commands, (0, 0))
pos2 = reduce(step2, commands, (0, 0, 0))
print(f'Answer 1: {pos1[0] * pos1[1]}\nAnswer 2: {pos2[0] * pos2[1]}')

Answer 1: 1728414
Answer 2: 1765720035


In [3]:
# DAY 03
def get_most_least_common(nums, pos):
    n0 = 0
    n1 = 0
    for num in nums:
        n0 += num[pos] == '0'
        n1 += num[pos] == '1'
    return ('1', '0') if n1 >= n0 else ('0', '1')

def part1(nums):
    lc, mc = "", ""
    for i in range(0, len(nums[0])):
        (m, l) = get_most_least_common(nums, i)
        mc += m
        lc += l
    return int(mc, 2) * int(lc, 2)
     
def part2(nums):
    mnums = nums[:]
    lnums = nums[:]
    for i in range(0, len(nums[0])):
        if len(mnums) > 1:
            (m, l) = get_most_least_common(mnums, i)
            mnums = [x for x in mnums if x[i] == m]

        if len(lnums) > 1:
            (m, l) = get_most_least_common(lnums, i)
            lnums = [x for x in lnums if x[i] == l]

    return int(mnums[0], 2) * int(lnums[0], 2)

nums = [x.strip() for x in open('../data/03.txt').readlines()]
print(f'Answer 1: {part1(nums)}\nAnswer 2: {part2(nums)}')

Answer 1: 2972336
Answer 2: 3368358


In [96]:
# DAY 04
def get_boards(data):
    cells = []
    for line in data:
        if line == '':
            yield cells
            cells = []
        else:
            cells.append([int(x) for x in line.split()])
    yield cells

def is_full(board, pos, is_vert):
    for i in range(len(board)):
        n = board[i][pos] if is_vert else board[pos][i]
        if n != -1:
            return False
    return True

def has_winner(board):
    return any(map(lambda i: 
                       is_full(board, i, True) or is_full(board, i, False), 
                   range(len(board))))

def apply_num(board, num):
    for row in board:
        for i, el in enumerate(row):
            if el == num:
                row[i] = -1
            
def run_simulation(nums, boards):
    winners, scores = [], []
    for n in nums:
        for i, board in enumerate(boards):
            if i in winners:
                continue
            apply_num(board, n)
            if has_winner(board):
                s = n * sum(x for row in board for x in row if x != -1)
                winners.append(i)
                scores.append(s)
    return scores
             
lines = [x.strip() for x in open('../data/04.txt').readlines()]
nums = [int(x) for x in lines[0].split(",")]
boards = list(get_boards(lines[2:]))
scores = run_simulation(nums, boards)

print(f"Answer 1: {scores[0]}\nAnswer 2: {scores[-1]}")

Answer 1: 44736
Answer 2: 1827


In [9]:
# DAY 05
from collections import defaultdict

def sgn(x):
    if x == 0:
        return 0
    return 1 if x > 0 else -1

def stroke(a, b, ptmap, skip_diagonals):
    (x1, y1) = a
    (x2, y2) = b
    dx = sgn(x2 - x1)
    dy = sgn(y2 - y1)
    
    if skip_diagonals and dx != 0 and dy != 0:
        return
    
    cx, cy = x1, y1
    while True:
        ptmap[(cx, cy)] += 1
        if cx == x2 and cy == y2:
            break
        cx += dx
        cy += dy
        
def find_num_overlaps(points, skip_diagonals):
    ptmap = defaultdict(int)
    for a, b in points:
        stroke(a, b, ptmap, skip_diagonals)
    return sum(ptmap[x] >= 2 for x in ptmap)

def parse_pt(pt):
    (x, y) = [int(c) for c in pt.split(',')]
    return (x, y)

points = [list(map(parse_pt, x.strip().split(' -> ')))
          for x in open('../data/05.txt').readlines()]
res1 = find_num_overlaps(points, True)
res2 = find_num_overlaps(points, False)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 7436
Answer 2: 21104


In [387]:
# DAY 06
def sum_spawned(nums, total_days):
    spawned = {}

    def get_n_spawned(n, days):
        if days <= n:
            return 0
        if (n, days) in spawned:
            return spawned[(n, days)]
        res = (days - n - 1) // 7 + 1

        for i in range(res):
            res += get_n_spawned(8, days - n - i * 7 - 1)
        spawned[(n, days)] = res
        return res

    return sum(get_n_spawned(x, total_days) for x in nums) + len(nums)

nums = [int(x) for x in open('../data/06.txt').read().split(',')]
print(f"Answer 1: {sum_spawned(nums, 80)}\nAnswer 2: {sum_spawned(nums, 256)}")

Answer 1: 386755
Answer 2: 1732731810807


In [19]:
# DAY 07
import statistics

def part1(nums):
    c = statistics.median(nums)
    return sum(abs(x - c) for x in nums)

# simple gradient descent
def fmin(p0, f, num_it=1000):
    EPS = 0.001
    p = p0
    for i in range(num_it):
        pr = f(p + EPS)
        pl = f(p - EPS)
        dp = (pr - pl)/(2 * EPS)
        p = p - dp * EPS
    return p

def part2(nums):
    def f(x):
        res = 0
        for n in nums:
            d = abs(x - n)
            res += d * (d + 1)/2
        return res
    minx = int(round(fmin(statistics.median(nums), f)))
    return f(minx)

nums = [int(x) for x in open('../data/07.txt').read().split(',')]
print(f"Answer 1: {int(part1(nums))}\nAnswer 2: {int(part2(nums))}")

Answer 1: 343468
Answer 2: 96086265
